In [16]:
#TODO: a python script version of this code will be coming out when I finish fine-tuning its features

#This program uses JGI IMG to create a list of gene product names associated with different pathways in a microbe sample.
#The list of product names are then used to create a new excel spreadsheet with original data rearranged by pathways
##########################################################################################################################
#You have to make sure that you have the pandas, splinter and BeautifulSoup libraries in order to run the program
from splinter import Browser
from bs4 import BeautifulSoup
import csv
from collections import OrderedDict
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from openpyxl import load_workbook
import numpy as np
from collections import Counter

In [17]:
#Opens up a Chrome browser, the file name depends on where your chromedriver is located
executable_path = {'executable_path':'C:\Program Files\chromedriver.exe'}
browser = Browser('chrome', **executable_path)

In [21]:
#visits the JGI link with the list of pathways, update the url link depending on your project
gene_url2 = "https://img.jgi.doe.gov/cgi-bin/mer/main.cgi?section=TaxonDetail&page=kegg&cat=cat&taxon_oid=2622736421"
browser.visit(gene_url2)
soup2 = BeautifulSoup(browser.html, "html.parser")

In [25]:
#find all the clickable links in the html file and add it the urlList
urlList = []
table = soup2.table
counter = 0
for a in table.find_all('a', href=True):
    if counter % 2 == 0: #the %2 is there to account for the fact that each pathway/row has two links and I only want one
        urlList.append(a['href'])
    counter += 1;

In [34]:
#build a list of pathways with each pathway containing a list of the gene product names
def hasNumbers(inputString):
    """
    Checks if the inputString has numbers in it
    """
    return any(char.isdigit() for char in inputString)

pathways = [] #array of the pathways with list of gene product names

for url in urlList[0:]:
    #visit each url and copy the gene product names 
    browser.visit("https://img.jgi.doe.gov/cgi-bin/mer/" + url)
    mainlst = browser.find_by_name('mainForm')
    soup = BeautifulSoup(mainlst.html, "html.parser")
    
    #get text from the html and convert into a list of words
    text = soup.find("p").get_text()
    text = text.replace("\xa0", "")
    test = text.split('\n')
    
    # find all the approriate pathway names (or those with numbers in them)
    pathway = [] #the array of gene product names for a given a pathway with the pathway name as the 1st element
    forfirst = 0
    for elm in test:
        if not elm.isspace() and not elm is '':
            if hasNumbers(elm[0:11]):
                #it is part of a pathway
                pathway.append(elm[11:].lstrip())
            else:
                if forfirst == 0:
                    pathway.append(elm)
                    forfirst = 1
                else:
                    #make append the old pathway to the pathways array and make a new pathway
                    pathways.append(pathway)
                    pathway = []
                    pathway.append(elm)
    pathways.append(pathway)

['Longevity regulating pathway - mammal', 'catalase', 'catalase', 'catalase', 'catalase', 'superoxide dismutase, Fe-Mn family']


In [27]:
#save data into a csv file
csvfile = "pathways.csv"

#Write each pathway as a row in the csv file
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(pathways)

In [2]:
#getting the data from pathways.csv and saving it into another pathways2 array
pathways2 = []
with open(csvfile) as csvDataFile:
        csvReader = csv.reader(csvDataFile)
        for row in csvReader:
            pathways2.append(row)
#removing duplicates in pathways to make it easier to arrange the old data by pathway
for index, pathway in enumerate(pathways2):
    pathways2[index] = list(OrderedDict.fromkeys(pathway))

['Longevity regulating pathway - mammal', 'catalase', 'catalase', 'catalase', 'catalase', 'superoxide dismutase, Fe-Mn family']


['Longevity regulating pathway - mammal',
 'catalase',
 'superoxide dismutase, Fe-Mn family']

In [48]:
#now I am going to actively analyze the edgeAll excel file and re-arrange and transfer the data to the results spreadsheet
#don't forget to update the file names accordingly
df1 = pd.read_excel('/home/cassandra/Halo Data Anlysis/edgeAll_with_Product_Name.xlsx')
df2 = pd.read_excel('/home/cassandra/Halo Data Anlysis/results.xlsx')

Column headings:
Index(['gene ID', 'transcript ID', 'low1_1', 'low2_2', 'low3_3', 'high1_1',
       'high2_2', 'high3_3', 'logFC', 'logCPM', 'LR', 'PValue', 'FDR',
       'Gene Product Name'],
      dtype='object')
Index(['Pathway', 'log'], dtype='object')


In [36]:
#This section makes an array from the elements in the Gene Product Name column in df1, it also prints out its first 10 elements
list_proname = df1['Gene Product Name']
print(list_proname[0:10])

480                 superoxide dismutase, Fe-Mn family  
481    cell elongation-specific peptidoglycan D,D-tra...
482                         penicillin-binding protein  
483                               hypothetical protein  
484                large subunit ribosomal protein L33  
485              5-formyltetrahydrofolate cyclo-ligase  
486    rhomboid family peptidase. Serine peptidase. M...
487                       Uncharacterized protein YqgQ  
488                                        glucokinase  
489                               hypothetical protein  
Name: Gene Product Name, dtype: object
<class 'str'>


In [47]:
#This section makes an array from the elements in the logFC column in df1, it also prints out its first 10 elements
list_fc = df1['logFC']
print(list_fc[0:10])

480    0.279450
481   -0.557643
482    0.165016
483   -1.424636
484   -0.801628
485    0.645176
486   -1.064923
487    1.564773
488    0.738546
489    0.056293
Name: logFC, dtype: float64


In [50]:
name_col = [] #the first column in the results spreadsheet with the pathway and gene product names 
fc_col = [] #the second column in the results spreadsheet with the logFC value 

#go through each pathway in pathways2 and look for matches in list_proname
for index, pathway in enumerate(pathways2[0:]):
    #the first element of pathway is just the pathway name so just append it to name_col
    name_col.append(pathway[0])
    fc_col.append('') #must append an empty string so the rows for both columns match up
    #go through the other elements of pathway and check for matches in list_proname
    for index, elm in enumerate(pathway[1:]):
        for ind, name in enumerate(list_proname):
            if name.lstrip().rstrip() == elm.lstrip().rstrip():
                name_col.append(list_proname[ind].lstrip().rstrip())
                fc_col.append(list_fc[ind])
    #add an empty row after each pathway 
    name_col.append('')
    fc_col.append('')

In [51]:
#create new data frames from the arrays and write them into the results spreadsheet
# don't forget to update the file names accordingly
df2['Pathway'] = name_col
df2['log'] = fc_col
df2.to_excel("/home/cassandra/Halo Data Anlysis/results.xlsx")